In [1]:
import tensorflow as tf
from os import listdir
from os.path import isdir, join
from tensorflow.keras import layers, models
import numpy as np

In [2]:
dataset_path = 'D:\data_speech_commands_v0.02'
all_targets = [name for name in listdir(dataset_path) if isdir(join(dataset_path, name))]
print(all_targets)

['backward', 'bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'follow', 'forward', 'four', 'go', 'happy', 'house', 'learn', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'visual', 'wow', 'yes', 'zero', '_background_noise_']


In [3]:
feature_sets_path = r'D:\voice_recognition'
feature_sets_filename = 'all_targets_mfcc_sets.npz'
model_filename = 'wake_word_stop_model.h5'
wake_word = 'stop'

In [4]:
feature_sets = np.load(join(feature_sets_path, feature_sets_filename))
x_train = feature_sets['x_train']
y_train = feature_sets['y_train']
x_val = feature_sets['x_val']
y_val = feature_sets['y_val']
x_test = feature_sets['x_test']
y_test = feature_sets['y_test']

In [5]:
wake_word_index1 = all_targets.index(wake_word)
y_train = np.equal(y_train, wake_word_index1).astype('float64')
y_val = np.equal(y_val, wake_word_index1).astype('float64')
y_test = np.equal(y_test, wake_word_index1).astype('float64')

In [6]:
print(y_val)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [7]:
x_train = x_train.reshape(x_train.shape[0], 
                          x_train.shape[1], 
                          x_train.shape[2], 
                          1)
x_val = x_val.reshape(x_val.shape[0], 
                      x_val.shape[1], 
                      x_val.shape[2], 
                      1)
x_test = x_test.reshape(x_test.shape[0], 
                        x_test.shape[1], 
                        x_test.shape[2], 
                        1)
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(7717, 16, 16, 1)
(972, 16, 16, 1)
(957, 16, 16, 1)


In [8]:
sample_shape = x_test.shape[1:]
print(sample_shape)

(16, 16, 1)


In [9]:
model = models.Sequential()
model.add(layers.Conv2D(32, 
                        (2, 2), 
                        activation='relu',
                        input_shape=sample_shape))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(32, (2, 2), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(64, (2, 2), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Classifier
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

In [10]:
model.compile(loss='binary_crossentropy', 
              optimizer='rmsprop', 
              metrics=['acc'])

In [11]:
with tf.device("/gpu:0"):
    history = model.fit(x_train, 
                        y_train, 
                        epochs=30, 
                        batch_size=100, 
                        validation_data=(x_val, y_val))

Train on 7717 samples, validate on 972 samples
Epoch 1/30
7717/7717 [==============================] - 3s 398us/sample - loss: 0.2227 - acc: 0.9480 - val_loss: 0.1317 - val_acc: 0.9681
Epoch 2/30
7717/7717 [==============================] - 0s 47us/sample - loss: 0.1485 - acc: 0.9623 - val_loss: 0.1131 - val_acc: 0.9681
Epoch 3/30
7717/7717 [==============================] - 0s 46us/sample - loss: 0.1296 - acc: 0.9629 - val_loss: 0.0913 - val_acc: 0.9681
Epoch 4/30
7717/7717 [==============================] - 0s 45us/sample - loss: 0.1157 - acc: 0.9657 - val_loss: 0.0921 - val_acc: 0.9681
Epoch 5/30
7717/7717 [==============================] - 0s 46us/sample - loss: 0.1044 - acc: 0.9668 - val_loss: 0.1605 - val_acc: 0.9414
Epoch 6/30
7717/7717 [==============================] - 0s 49us/sample - loss: 0.1002 - acc: 0.9676 - val_loss: 0.0963 - val_acc: 0.9712
Epoch 7/30
7717/7717 [==============================] - 0s 48us/sample - loss: 0.0917 - acc: 0.9701 - val_loss: 0.1149 - val_acc: 